In [2]:
# Synthetic Dataset Generator for Testing
# This notebook creates real and synthetic dataset pairs for testing the quality rater app

import pandas as pd
import numpy as np
from sdv.metadata import SingleTableMetadata
from sdv.single_table import GaussianCopulaSynthesizer, CTGANSynthesizer, TVAESynthesizer
import os

print("🎯 Synthetic Dataset Generator for Quality Rater App")
print("=" * 60)

# ============================================================================
# Dataset 1: Customer Banking Data (Medium Complexity)
# ============================================================================
print("\n📊 Dataset 1: Customer Banking Data")

np.random.seed(42)
n_customers = 2000

# Generate realistic customer data with correlations
age = np.random.normal(45, 15, n_customers).clip(18, 85)
income = 20000 + (age - 18) * 1200 + np.random.normal(0, 15000, n_customers)
income = income.clip(20000, 300000)

credit_score = 300 + (income / 1000) * 1.5 + np.random.normal(0, 50, n_customers)
credit_score = credit_score.clip(300, 850)

# Premium customers have higher income and credit scores
is_premium = (income > 75000) & (credit_score > 700)
is_premium = is_premium | (np.random.random(n_customers) < 0.1)  # 10% random premium

account_balance = np.where(is_premium, 
                           np.random.exponential(15000, n_customers),
                           np.random.exponential(3000, n_customers))

num_transactions = np.random.poisson(20 + (is_premium * 15), n_customers)

regions = np.random.choice(['North', 'South', 'East', 'West'], n_customers)
segments = np.where(is_premium, 
                    np.random.choice(['Gold', 'Platinum'], n_customers),
                    np.random.choice(['Bronze', 'Silver'], n_customers))

banking_data = pd.DataFrame({
    'customer_id': range(1, n_customers + 1),
    'age': age.astype(int),
    'income': income.round(2),
    'credit_score': credit_score.round(0).astype(int),
    'account_balance': account_balance.round(2),
    'num_transactions': num_transactions,
    'is_premium': is_premium.astype(int),
    'region': regions,
    'customer_segment': segments
})

# Save real dataset
banking_data.to_csv('banking_real.csv', index=False)
print(f"✅ Saved: banking_real.csv ({len(banking_data)} rows, {len(banking_data.columns)} columns)")

# Generate synthetic version using GaussianCopula
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(banking_data)

synthesizer = GaussianCopulaSynthesizer(metadata)
synthesizer.fit(banking_data)
banking_synthetic = synthesizer.sample(num_rows=len(banking_data))
banking_synthetic.to_csv('banking_synthetic.csv', index=False)
print(f"✅ Saved: banking_synthetic.csv (GaussianCopula)")

# ============================================================================
# Dataset 2: E-commerce Transactions (High Complexity)
# ============================================================================
print("\n🛒 Dataset 2: E-commerce Transactions")

np.random.seed(123)
n_transactions = 5000

# Generate transaction data with temporal patterns
days = np.random.randint(1, 366, n_transactions)
is_weekend = np.isin(days % 7, [0, 6])
is_holiday = np.isin(days, [1, 365, 180, 270])  # Major holidays

# Higher transaction amounts on weekends/holidays
base_amount = np.random.gamma(2, 30, n_transactions)
amount = np.where(is_weekend | is_holiday, base_amount * 1.5, base_amount)
amount = amount.clip(5, 500)

# Customer types affect behavior
customer_type = np.random.choice(['New', 'Regular', 'VIP'], n_transactions, p=[0.3, 0.5, 0.2])
discount_pct = np.where(customer_type == 'VIP', 
                        np.random.uniform(10, 30, n_transactions),
                        np.where(customer_type == 'Regular',
                                np.random.uniform(0, 15, n_transactions),
                                np.random.uniform(0, 5, n_transactions)))

items_purchased = np.where(customer_type == 'VIP',
                           np.random.poisson(5, n_transactions),
                           np.random.poisson(2, n_transactions)) + 1

payment_method = np.where(amount > 100,
                         np.random.choice(['Credit', 'Debit', 'PayPal'], n_transactions, p=[0.6, 0.2, 0.2]),
                         np.random.choice(['Credit', 'Debit', 'PayPal'], n_transactions, p=[0.3, 0.5, 0.2]))

categories = np.random.choice(['Electronics', 'Clothing', 'Home', 'Sports', 'Books'], n_transactions)

ecommerce_data = pd.DataFrame({
    'transaction_id': range(1, n_transactions + 1),
    'day_of_year': days,
    'amount': amount.round(2),
    'discount_pct': discount_pct.round(1),
    'items_purchased': items_purchased,
    'customer_type': customer_type,
    'payment_method': payment_method,
    'category': categories,
    'is_weekend': is_weekend.astype(int),
    'is_holiday': is_holiday.astype(int)
})

# Save real dataset
ecommerce_data.to_csv('ecommerce_real.csv', index=False)
print(f"✅ Saved: ecommerce_real.csv ({len(ecommerce_data)} rows, {len(ecommerce_data.columns)} columns)")

# Generate synthetic version
metadata2 = SingleTableMetadata()
metadata2.detect_from_dataframe(ecommerce_data)

synthesizer2 = GaussianCopulaSynthesizer(metadata2)
synthesizer2.fit(ecommerce_data)
ecommerce_synthetic = synthesizer2.sample(num_rows=len(ecommerce_data))
ecommerce_synthetic.to_csv('ecommerce_synthetic.csv', index=False)
print(f"✅ Saved: ecommerce_synthetic.csv (GaussianCopula)")

# ============================================================================
# Dataset 3: Medical Patient Records (Complex with Missing Data)
# ============================================================================
print("\n🏥 Dataset 3: Medical Patient Records")

np.random.seed(789)
n_patients = 1500

patient_age = np.random.normal(55, 20, n_patients).clip(18, 95).astype(int)
gender = np.random.choice(['M', 'F'], n_patients)

# Correlated health metrics
bmi = np.random.normal(27, 5, n_patients).clip(15, 50)
blood_pressure_sys = 90 + (bmi - 20) * 2 + np.random.normal(0, 15, n_patients)
blood_pressure_sys = blood_pressure_sys.clip(80, 200)

cholesterol = 150 + (bmi - 20) * 5 + np.random.normal(0, 30, n_patients)
cholesterol = cholesterol.clip(100, 400)

# Diagnosis based on risk factors
high_risk = (bmi > 30) | (blood_pressure_sys > 140) | (cholesterol > 240)
diagnosis = np.where(high_risk,
                    np.random.choice(['Hypertension', 'Diabetes', 'Heart Disease'], n_patients),
                    np.random.choice(['Healthy', 'Mild Risk'], n_patients, p=[0.7, 0.3]))

num_visits = np.where(high_risk, 
                     np.random.poisson(8, n_patients),
                     np.random.poisson(2, n_patients)) + 1

medications = np.where(high_risk,
                      np.random.randint(1, 5, n_patients),
                      np.random.randint(0, 2, n_patients))

# Add some missing values (realistic for medical data)
glucose = np.random.normal(100, 25, n_patients)
glucose = np.where(np.random.random(n_patients) < 0.15, np.nan, glucose)  # 15% missing

medical_data = pd.DataFrame({
    'patient_id': range(1, n_patients + 1),
    'age': patient_age,
    'gender': gender,
    'bmi': bmi.round(1),
    'blood_pressure_systolic': blood_pressure_sys.round(0),
    'cholesterol': cholesterol.round(0),
    'glucose': glucose,
    'diagnosis': diagnosis,
    'num_visits': num_visits,
    'medications': medications
})

# Save real dataset
medical_data.to_csv('medical_real.csv', index=False)
print(f"✅ Saved: medical_real.csv ({len(medical_data)} rows, {len(medical_data.columns)} columns)")
print(f"   ⚠️  Note: Contains {medical_data['glucose'].isna().sum()} missing values")

# Generate synthetic version
metadata3 = SingleTableMetadata()
metadata3.detect_from_dataframe(medical_data)

synthesizer3 = GaussianCopulaSynthesizer(metadata3)
synthesizer3.fit(medical_data)
medical_synthetic = synthesizer3.sample(num_rows=len(medical_data))
medical_synthetic.to_csv('medical_synthetic.csv', index=False)
print(f"✅ Saved: medical_synthetic.csv (GaussianCopula)")

# ============================================================================
# Dataset 4: Simple Test Case (Low Complexity - Should Score High)
# ============================================================================
print("\n🎯 Dataset 4: Simple Test Case")

np.random.seed(999)
n_simple = 1000

simple_data = pd.DataFrame({
    'id': range(1, n_simple + 1),
    'value_a': np.random.normal(100, 15, n_simple),
    'value_b': np.random.exponential(50, n_simple),
    'category': np.random.choice(['A', 'B', 'C'], n_simple),
    'flag': np.random.choice([0, 1], n_simple, p=[0.7, 0.3])
})

simple_data.to_csv('simple_real.csv', index=False)
print(f"✅ Saved: simple_real.csv ({len(simple_data)} rows, {len(simple_data.columns)} columns)")

# Generate high-quality synthetic
metadata4 = SingleTableMetadata()
metadata4.detect_from_dataframe(simple_data)

synthesizer4 = GaussianCopulaSynthesizer(metadata4)
synthesizer4.fit(simple_data)
simple_synthetic = synthesizer4.sample(num_rows=len(simple_data))
simple_synthetic.to_csv('simple_synthetic.csv', index=False)
print(f"✅ Saved: simple_synthetic.csv (GaussianCopula)")

# ============================================================================
# Summary
# ============================================================================
print("\n" + "=" * 60)
print("✨ Dataset Generation Complete!")
print("=" * 60)
print("\n📁 Generated Test Datasets:")
print("   1. Banking Data:     banking_real.csv + banking_synthetic.csv")
print("   2. E-commerce:       ecommerce_real.csv + ecommerce_synthetic.csv")
print("   3. Medical Records:  medical_real.csv + medical_synthetic.csv")
print("   4. Simple Test:      simple_real.csv + simple_synthetic.csv")
print("\n💡 Usage:")
print("   - Upload any *_real.csv as 'Original dataset'")
print("   - Upload matching *_synthetic.csv as 'Synthetic dataset'")
print("   - Compare quality scores across different dataset types")
print("\n📊 Expected Quality Scores:")
print("   - Simple Test:       0.85-0.95 (should score high)")
print("   - Banking Data:      0.70-0.85 (good quality)")
print("   - E-commerce:        0.65-0.80 (moderate complexity)")
print("   - Medical Records:   0.60-0.75 (handles missing data)")
print("\n🎯 All datasets saved to current directory")

🎯 Synthetic Dataset Generator for Quality Rater App

📊 Dataset 1: Customer Banking Data
✅ Saved: banking_real.csv (2000 rows, 9 columns)


d:\Documents\data_rater\.venv\Lib\site-packages\sdv\single_table\base.py:168: FutureWarning:

The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.

d:\Documents\data_rater\.venv\Lib\site-packages\sdv\single_table\base.py:134: UserWarning:

We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.



✅ Saved: banking_synthetic.csv (GaussianCopula)

🛒 Dataset 2: E-commerce Transactions
✅ Saved: ecommerce_real.csv (5000 rows, 10 columns)


d:\Documents\data_rater\.venv\Lib\site-packages\sdv\single_table\base.py:168: FutureWarning:

The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.

d:\Documents\data_rater\.venv\Lib\site-packages\sdv\single_table\base.py:134: UserWarning:

We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.



✅ Saved: ecommerce_synthetic.csv (GaussianCopula)

🏥 Dataset 3: Medical Patient Records
✅ Saved: medical_real.csv (1500 rows, 10 columns)
   ⚠️  Note: Contains 233 missing values


d:\Documents\data_rater\.venv\Lib\site-packages\sdv\single_table\base.py:168: FutureWarning:

The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.

d:\Documents\data_rater\.venv\Lib\site-packages\sdv\single_table\base.py:134: UserWarning:

We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.



✅ Saved: medical_synthetic.csv (GaussianCopula)

🎯 Dataset 4: Simple Test Case
✅ Saved: simple_real.csv (1000 rows, 5 columns)


d:\Documents\data_rater\.venv\Lib\site-packages\sdv\single_table\base.py:168: FutureWarning:

The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.

d:\Documents\data_rater\.venv\Lib\site-packages\sdv\single_table\base.py:134: UserWarning:

We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.



✅ Saved: simple_synthetic.csv (GaussianCopula)

✨ Dataset Generation Complete!

📁 Generated Test Datasets:
   1. Banking Data:     banking_real.csv + banking_synthetic.csv
   2. E-commerce:       ecommerce_real.csv + ecommerce_synthetic.csv
   3. Medical Records:  medical_real.csv + medical_synthetic.csv
   4. Simple Test:      simple_real.csv + simple_synthetic.csv

💡 Usage:
   - Upload any *_real.csv as 'Original dataset'
   - Upload matching *_synthetic.csv as 'Synthetic dataset'
   - Compare quality scores across different dataset types

📊 Expected Quality Scores:
   - Simple Test:       0.85-0.95 (should score high)
   - Banking Data:      0.70-0.85 (good quality)
   - E-commerce:        0.65-0.80 (moderate complexity)
   - Medical Records:   0.60-0.75 (handles missing data)

🎯 All datasets saved to current directory
